# Jacobian

Lets begin with some definitions

$$
J_{IK} = \frac{\partial x}{\partial(rs)} = \frac{\partial N_i}{\partial(\eta_K)} X_iI
$$

being $X_iI$ the nodal coordinates in the $I$ dimension, the current position and X the reference position

In [3]:
import numpy as np

In [10]:
import numpy as np
# Define material properties
E = 200e9  # Young's modulus in Pa
nu = 0.3   # Poisson's ratio
m_dim = 2
m_nodxelem = 4
# Define element properties
num_nodes_element = 4  # Number of nodes per element
element_length = 1.0   # Length of the element

dNdX = np.zeros((m_dim, m_nodxelem)) 
# Define shape functions and their derivatives for 2D quadrilateral element
def shape_functions(xi, eta):
    N = np.array([(1-xi)*(1-eta)/4,
                  (1+xi)*(1-eta)/4,
                  (1+xi)*(1+eta)/4,
                  (1-xi)*(1+eta)/4])
    dNdX[0,:] = np.array([-(1-eta)/4, (1-eta)/4, (1+eta)/4, -(1+eta)/4])
    dNdX[1,:] = np.array([-(1-xi)/4, -(1+xi)/4, (1+xi)/4, (1-xi)/4])
    return N, dNdX
    print(dNdX)
# Define material matrix for plane stress
def material_matrix():
    C = E / (1 - nu**2) * np.array([[1, nu, 0],
                                     [nu, 1, 0],
                                     [0, 0, (1 - nu) / 2]])
    return C

# Gauss quadrature points and weights
gauss_points = np.array([[-0.577350269, -0.577350269],
                         [ 0.577350269, -0.577350269],
                         [ 0.577350269,  0.577350269],
                         [-0.577350269,  0.577350269]])
gauss_weights = np.array([1, 1, 1, 1])

gp_count = len(gauss_points)

In case of elasticity, we can define $ B^T C B$

In case of elasticity, we can define $ B^T C B$

In [17]:
# Finite element strain rate calculation
def calculate_jacobian(velocities):
    strain_rate = np.zeros((3, 3))
    J = np.zeros((gp_count, 2, 2))
    for gp in range(len(gauss_points)):
        xi, eta = gauss_points[gp]
        weight = gauss_weights[gp]
        N, dN_dX = shape_functions(xi, eta)
        J[gp] = np.dot(dNdX, velocities)
        detJ = np.linalg.det(J[gp])
        invJ = np.linalg.inv(J[gp])
    return J


In [24]:
# Finite element strain rate calculation
def calculate_strain_rate(velocities):
    strain_rate = np.zeros((3, 3))
    J = np.zeros((gp_count, 2, 2))
    for gp in range(len(gauss_points)):
        J = calculate_jacobian(velocities)
        xi, eta = gauss_points[gp]
        weight = gauss_weights[gp]
        N, dNdX = shape_functions(xi, eta)
        J[gp] = np.dot(dNdX, velocities)
        detJ = np.linalg.det(J[gp])
        invJ = np.linalg.inv(J[gp])
        B = np.zeros((3, 8))
        for i in range(num_nodes_element):
            B[0, 2*i] = dN_dxi[i]
            B[1, 2*i+1] = dN_deta[i]
            B[2, 2*i] = dN_deta[i]
            B[2, 2*i+1] = dN_dxi[i]
        C = material_matrix()
        #strain_rate += np.dot(B.T, np.dot(invJ.T, np.dot(B, C))) * detJ * weight
        print ("Jacobian", J[gp])
    strain_rate *= 0.5
    return strain_rate

Now use it 

In [25]:
# Example usage
x    =  np.array([[0., 0.], [0.1, 0.], [0., 0.1], [0.1, 0.1]])
velocities = np.array([[1, 0], [2, 0], [2, 1], [1, 1]])  # Example velocities at nodes
strain_rate = calculate_strain_rate(velocities)
print("Strain rate:")
print(strain_rate)

NameError: name 'dN_dxi' is not defined

## Velocity Gradient and Strain Rate

In [ ]:
# Define nodal velocities (dummy data for demonstration)
nodal_velocities = np.full(24, 0.1)

def velocity_gradient_tensor(dNdX, nodal_velocities):
    grad_v = np.zeros((3, 3))
    for I in range(3):
        for J in range(3):
            for a in range(8):
                grad_v[I, J] += dNdX[I, a] * nodal_velocities[J * 8 + a]
    return grad_v